In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)
    
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.calculation.work import WorkCalculation
from aiida.orm.calculation.job.quantumespresso.pw import PwCalculation

import urlparse
from base64 import b64encode
from StringIO import StringIO
from IPython.display import Javascript

In [ ]:
url = urlparse.urlsplit(jupyter_notebook_url)
pk = int(urlparse.parse_qs(url.query)['pk'][0])

qb = QueryBuilder()
qb.append(WorkCalculation, filters={'id':pk})
qb.append(PwCalculation, output_of=WorkCalculation, filters={'label':'scf'})
assert qb.count() == 1
scfcalc = qb.first()[0]

structure = scfcalc.inp.structure
ase_struct = structure.get_ase()
f = StringIO()
ase_struct.write(f, format="xyz")
xyzdata = b64encode(f.getvalue())
fn = ase_struct.get_chemical_formula() + "_pk%d.xyz" % structure.pk

javascript  = 'var fileContents="%s";\n' % xyzdata
javascript += 'var link = document.createElement("a");\n'
javascript += 'link.download = "%s";\n' % fn
javascript += 'link.href = "data:chemical/x-xyz;base64," + fileContents;\n'
javascript += 'link.click();'
javascript += 'window.close();'

display(Javascript(javascript))